In [96]:
import os
import django
import pandas as pd
import numpy as np
import datetime
from django.db import transaction
from django.utils import timezone
from pathlib import Path
from django.db.models import Max

# os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend.settings")
# os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# django.setup()

# from mainApp.models import Field, FieldByYear, Course, Semester, Faculty
import sqlite3

In [97]:
faculty_df = pd.read_csv("sylabus_data/faculty_data_2025_1_stopien.csv", sep=';', index_col=0)
course_df = pd.read_csv("sylabus_data/course_data_2025_1_stopien.csv", sep=';', index_col=0)
field_df = pd.read_csv("sylabus_data/field_data_2025_1_stopien.csv", sep=';', index_col=0)

In [98]:
faculty_df.head()

,name
0,Wydział Inżynierii Lądowej i Gospodarki Zasobami
1,Wydział Inżynierii Metali i Informatyki Przemy...
2,"Wydział Elektrotechniki, Automatyki, Informaty..."
3,"Wydział Informatyki, Elektroniki i Telekomunik..."
4,Wydział Inżynierii Mechanicznej i Robotyki


In [99]:
field_df.head()

,faculty_id,name,kind,type,description,specialization
0,0,Budownictwo,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Budownictwo należy do obszaru...,NaN
1,0,Budownictwo,Studia inżynierskie I stopnia,Niestacjonarne,Kierunek studiów Budownictwo należy do obszaru...,NaN
2,0,Inżynieria Górnicza,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,NaN
3,0,Inżynieria Górnicza,Studia inżynierskie I stopnia,Niestacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,NaN
4,0,Inżynieria i Zarządzanie Procesami Przemysłowymi,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Inżynieria i Zarządzanie Proc...,NaN


In [100]:
course_df.head()

,field_id,name,semester,ects,test_type,lecturer,description
0,0,Geologia,1,3.0,Zaliczenie,Katarzyna Cyran,Zajęcia prowadzone w trybie mieszanym. Deforma...
1,0,Geometria wykreślna,1,2.0,Zaliczenie,Krzysztof Pałac,Ćwiczenia polegają na samodzielnym wykonywaniu...
2,0,Grafika inżynierska i rysunek techniczny,1,3.0,Zaliczenie,Sebastian Olesiak,Rysunek techniczny z zastosowaniem tradycyjnyc...
3,0,Geodezja,1,3.0,Zaliczenie,Michał Strach,1. Podstawowe pojęcia geodezyjne. Organizacja ...
4,0,Fizyka I,1,3.0,Zaliczenie,Michał Ślęzak,Wykład: Głównym celem wykładów jest jakościowe...


### Mapowanie nazw wydziałów (id) na nowe (id) w tabeli field w przypadku różnic w bazie i pliku

In [101]:
cnx = sqlite3.connect('db.sqlite3')
sql_faculty_df = pd.read_sql_query("SELECT * FROM mainApp_faculty", cnx, index_col='id').iloc[:-2]

In [102]:
faculty_map = dict(zip(faculty_df.index.values, faculty_df.name))

faculty_df = pd.concat([sql_faculty_df, faculty_df])
faculty_df = faculty_df.drop_duplicates('name')
faculty_df.index = np.arange(1, len(faculty_df)+1)
faculty_df = faculty_df.fillna(1)

new_faculty_map = {}
for key, val in faculty_map.items():
    new_faculty_map[key] = faculty_df.index[faculty_df['name'] == val].tolist()[0]

field_df['faculty_id'] = field_df['faculty_id'].replace(new_faculty_map)
field_df['specialization'] = field_df['specialization'].fillna('')

field_df.head()

,faculty_id,name,kind,type,description,specialization
0,16,Budownictwo,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Budownictwo należy do obszaru...,
1,16,Budownictwo,Studia inżynierskie I stopnia,Niestacjonarne,Kierunek studiów Budownictwo należy do obszaru...,
2,16,Inżynieria Górnicza,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,
3,16,Inżynieria Górnicza,Studia inżynierskie I stopnia,Niestacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,
4,16,Inżynieria i Zarządzanie Procesami Przemysłowymi,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Inżynieria i Zarządzanie Proc...,


### Mapowanie nazw kierunków (id) na nowe (id) w tabeli course w przypadku różnic w bazie i pliku

In [103]:
sql_field_df = pd.read_sql_query("SELECT * FROM mainApp_field", cnx, index_col='id')
sql_field_df = sql_field_df.drop_duplicates(['name', 'type'])
sql_field_df[['description', 'specialization']] = sql_field_df[['description', 'specialization']].fillna('')
sql_faculty_df['faculty_id'] = None
sql_faculty_df['level'] = None
sql_field_df.head()

,name,formula,description,specialization,type
id,,,,,
107,Budownictwo,2*M+3*G1+G2,,,stacjonarne
108,Inżynieria Górnicza,2*M+3*G1+G2,,,stacjonarne
109,Inżynieria Kształtowania Środowiska,2*M+3*G1+G2,,,stacjonarne
110,Inżynieria i Zarządzanie Procesami Przemysłowymi,2*M+3*G1+G2,,,stacjonarne
111,Rewitalizacja Terenów Zdegradowanych,2*M+3*G1+G2,,,stacjonarne


In [104]:
field_level_map = {'Studia inżynierskie I stopnia': 'I stopień', 'Studia licencjackie I stopnia': 'I stopień'}
field_df = field_df.rename(columns={'kind': 'level'})
field_df['level'] = field_df['level'].replace(field_level_map)
field_df['type'] = field_df['type'].str.lower()

field_map = dict(zip(field_df.index.values, field_df.name + ';' + field_df.level + ';' + field_df.type))
# field_map

merged = pd.merge(
    sql_field_df, field_df, 
    on=['name', 'type'], 
    how='outer', 
    suffixes=('_1', '_2'), 
    indicator=True
)

field_df = merged[['name', 'formula', 'type', 'faculty_id', 'level', 'description_2', 'specialization_2']]
field_df = field_df.fillna('')
field_df

,name,formula,type,faculty_id,level,description_2,specialization_2
0,Automatyka Przemysłowa i Robotyka,2*M+3*G1+G2,niestacjonarne,4.0,I stopień,"Biorąc pod uwagę wymagania rynku pracy, koncep...",
1,Automatyka Przemysłowa i Robotyka,2*M+3*G1+G2,stacjonarne,4.0,I stopień,"Biorąc pod uwagę wymagania rynku pracy, koncep...",
2,Automatyka i Robotyka,2*M+3*G1+G2,stacjonarne,2.0,I stopień,Celem kształcenia na kierunku Automatyka i Rob...,
3,Budownictwo,2*M+3*G1+G2,niestacjonarne,16.0,I stopień,Kierunek studiów Budownictwo należy do obszaru...,
4,Budownictwo,2*M+3*G1+G2,stacjonarne,16.0,I stopień,Kierunek studiów Budownictwo należy do obszaru...,
...,...,...,...,...,...,...,...
145,Zaawansowane Materiały Inżynierskie (sukcesor ...,2*M+3*G1+G2,stacjonarne,,,,
146,Zarządzanie,2*M+3*G1+G2,niestacjonarne,11.0,I stopień,Zarządzanie na Wydziale Zarządzania Akademii G...,
147,Zarządzanie,2*M+3*G1+G2,stacjonarne,11.0,I stopień,Zarządzanie na Wydziale Zarządzania Akademii G...,
148,Zarządzanie i Inżynieria Produkcji,2*M+3*G1+G2,niestacjonarne,11.0,I stopień,Zarządzanie i Inżynieria Produkcji na Wydziale...,


In [ ]:
new_field_map = {}
# Odwracamy słownik: teraz klucz to string, a wartość to indeks z oryginalnego słownika
for key, value in field_map.items():
    try:
        name, level, type_ = value.split(';')
        # Znajdź indeks w df, gdzie wszystkie trzy kolumny się zgadzają
        match = field_df[
            (field_df['name'] == name) & 
            (field_df['level'] == level) & 
            (field_df['type'] == type_)
        ]
        if not match.empty:
            new_field_map[key] = match.index[0]  # lub match['id'].values[0] jeśli id jest kolumną
    except ValueError:
        print(f"Niepoprawny format wartości: {value}")

course_df['field_id'] = course_df['field_id'].replace(new_field_map)

course_df.head()

,field_id,name,semester,ects,test_type,lecturer,description
0,4,Geologia,1,3.0,Zaliczenie,Katarzyna Cyran,Zajęcia prowadzone w trybie mieszanym. Deforma...
1,4,Geometria wykreślna,1,2.0,Zaliczenie,Krzysztof Pałac,Ćwiczenia polegają na samodzielnym wykonywaniu...
2,4,Grafika inżynierska i rysunek techniczny,1,3.0,Zaliczenie,Sebastian Olesiak,Rysunek techniczny z zastosowaniem tradycyjnyc...
3,4,Geodezja,1,3.0,Zaliczenie,Michał Strach,1. Podstawowe pojęcia geodezyjne. Organizacja ...
4,4,Fizyka I,1,3.0,Zaliczenie,Michał Ślęzak,Wykład: Głównym celem wykładów jest jakościowe...


## Podsumowanie

In [110]:
faculty_df.head()

,name,building_id
1,Wydział Inżynierii Metali i Informatyki Przemy...,1.0
2,"Wydział Elektrotechniki, Automatyki, Informaty...",1.0
3,"Wydział Informatyki, Elektroniki i Telekomunik...",1.0
4,Wydział Inżynierii Mechanicznej i Robotyki,1.0
5,"Wydział Geologii, Geofizyki i Ochrony Środowiska",1.0


In [111]:
field_df.head()

,name,formula,type,faculty_id,level,description_2,specialization_2
0,Automatyka Przemysłowa i Robotyka,2*M+3*G1+G2,niestacjonarne,4.0,I stopień,"Biorąc pod uwagę wymagania rynku pracy, koncep...",
1,Automatyka Przemysłowa i Robotyka,2*M+3*G1+G2,stacjonarne,4.0,I stopień,"Biorąc pod uwagę wymagania rynku pracy, koncep...",
2,Automatyka i Robotyka,2*M+3*G1+G2,stacjonarne,2.0,I stopień,Celem kształcenia na kierunku Automatyka i Rob...,
3,Budownictwo,2*M+3*G1+G2,niestacjonarne,16.0,I stopień,Kierunek studiów Budownictwo należy do obszaru...,
4,Budownictwo,2*M+3*G1+G2,stacjonarne,16.0,I stopień,Kierunek studiów Budownictwo należy do obszaru...,


In [114]:
field_df[field_df['faculty_id']==''].head() # Nie z pierwszego stopnia

,name,formula,type,faculty_id,level,description_2,specialization_2
6,Chemia Budowlana,2*M+3*G1+G2,stacjonarne,,,,
9,"Computer Science (płatny, w języku angielskim,...",2*M+3*G1+G2,stacjonarne,,,,
13,Edukacja Techniczno-Informatyczna,2*M+3*G1+G2,stacjonarne,,,,
17,Electronics and Telecommunications (w języku a...,2*M+3*G1+G2,stacjonarne,,,,
21,Elektronika i Telekomunikacja (w języku angiel...,2*M+3*G1+G2,stacjonarne,,,,


In [115]:
course_df.head()

,field_id,name,semester,ects,test_type,lecturer,description
0,4,Geologia,1,3.0,Zaliczenie,Katarzyna Cyran,Zajęcia prowadzone w trybie mieszanym. Deforma...
1,4,Geometria wykreślna,1,2.0,Zaliczenie,Krzysztof Pałac,Ćwiczenia polegają na samodzielnym wykonywaniu...
2,4,Grafika inżynierska i rysunek techniczny,1,3.0,Zaliczenie,Sebastian Olesiak,Rysunek techniczny z zastosowaniem tradycyjnyc...
3,4,Geodezja,1,3.0,Zaliczenie,Michał Strach,1. Podstawowe pojęcia geodezyjne. Organizacja ...
4,4,Fizyka I,1,3.0,Zaliczenie,Michał Ślęzak,Wykład: Głównym celem wykładów jest jakościowe...
